In [ ]:
import pandas as pd
import numpy as np
import copy
import pymrmr
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams["font.sans-serif"] = ["Times New Roman"] # change font as Times New Roman

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, auc, roc_auc_score, RocCurveDisplay

from collections import defaultdict
from rdkit import Chem
from rdkit import DataStructs

#### Dataset Input 数据读取

In [ ]:
path = "../Data/131 Exhaustion to PLA.xlsx"
nams = pd.read_excel(path, index_col = 0).index.values
typs = pd.read_excel(path, index_col = 0)["Type"]

azos = [ind for ind, typ in enumerate(typs) if typ == "单偶氮"]
anqs = [ind for ind, typ in enumerate(typs) if typ == "蒽醌"]
oths = [ind for ind, typ in enumerate(typs) if typ != "单偶氮" and typ != "蒽醌"]

In [ ]:
mac_fp = pd.read_csv("..\Data\model_maccs.csv").iloc[:,1:]
pub_fp = pd.read_csv("..\Data\model_pubchem.csv").iloc[:,1:] 
sub_fp = pd.read_csv("..\Data\model_substructure.csv").iloc[:,1:]
suc_fp = pd.read_csv("..\Data\model_substructure count.csv").iloc[:,1:]
est_fp = pd.read_csv("..\Data\model_estate.csv").iloc[:,1:]

sub_fp.index = est_fp.index = mac_fp.index = pub_fp.index = suc_fp.index = nams
esm_fp = pd.concat([mac_fp, pub_fp, sub_fp, suc_fp, est_fp], axis = 1)

exhs = pd.read_excel(path, index_col = 0)["Exhaustion"]
cate = [int(score//80) for score in exhs]

#### Dataset Pretreatment 数据预处理

In [ ]:
var = VarianceThreshold(threshold = 0)
esm_fp2 = var.fit_transform(esm_fp)
esm_fp2 = esm_fp.loc[:, var.get_support()]

corr_matrix = esm_fp2.corr(numeric_only = True).abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
esm_fp3 = esm_fp2.drop(to_drop, axis = 1)

In [ ]:
SKFold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
SKFold_5 =  StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
criterion = {"accuracy":"ACC", "f1_weighted":"F1", "roc_auc":"AUC"}

fp_tra, fp_tes, cate_tra, cate_tes = train_test_split(esm_fp3, cate, train_size = 0.8, random_state = 41, stratify = cate)

#### Function Definetion 定义函数

In [ ]:
# mRMR function 定义mRMR函数
def mrmr_selection(feature, target, num: int):
    mrmr_feas = []
    feature_mrmr = feature.copy()
    feature_mrmr.insert(loc = 0,column = "Category",value = target)
    mrmr_feas = pymrmr.mRMR(feature_mrmr, 'MIQ', num)
    return mrmr_feas

# Stratified cross-validation function 定义分层交叉验证性能函数
def skfold_perf(n, model, X_data, y_data):
    performance = []
    SKFold = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)  # N-fold  N折分层交叉验证
    criterion = {"accuracy":"ACC", "f1_weighted":"F1", "roc_auc":"AUC"}  # Validation metrics 评价标准参数
    for scoring in criterion:
        answer = cross_val_score(model, X_data, np.array(y_data).ravel(), cv = SKFold, scoring = scoring)
        performance.append(answer.mean().round(3))
    return performance

# Confusion matrix drawing function 定义混淆矩阵绘图函数
def cm_drawing(n:int, model, X_data, y_data:list): # N: fold of stratified cross-validation n:分层交叉验证折数
    X = X_data.astype("float")
    Y = np.array(y_data).ravel()
    
    predictions = defaultdict(list)  # 建立空字典记录
    
    SKFold = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)  # 交叉验证方法
    for train_index, test_index in SKFold.split(X, Y):  
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]  
        Y_train, Y_test = Y[train_index], Y[test_index] 
        model.fit(X_train, Y_train)  
        Y_pred = model.predict(X_test)  
        predictions['pred'].extend(Y_pred)  # record key and value 记录字典中键值对
        predictions['test'].extend(Y_test) 
        
    cm = confusion_matrix(predictions["test"], predictions["pred"])  # confusion matrix 混淆矩阵   
    sns.set_theme(style = "whitegrid",font = 'Times New Roman', font_scale = 2)  # drawing via seaborn seaborn绘制混淆矩阵
    plt.figure()  
    sns.heatmap(cm, annot = True, fmt = ".2g", cmap = "viridis", linewidths = 3, vmin = 0, vmax = 60,
                linecolor = "white", square = True)
    plt.xlabel("Predicted Class", fontsize = 20)  
    plt.ylabel("Experimental Class", fontsize = 20)  
    plt.show()

# ROC curve drawing function ROC曲线绘制函数
def cv_roc_drawing(n:int, model, X_data, y_data): # n 折数；model 训练完成的模型；X_data, y_data 数据
    tprs = []  # Recording True Positive Rate 记录真正率
    aucs = []  # Recording AUC value 记录AUC值
    mean_fpr = np.linspace(0, 1, 100)
    
    SKF = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)
    fig, ax = plt.subplots(figsize = (15, 15))
    for i, (train_index, test_index) in enumerate(SKF.split(X_data, y_data)):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]  
        Y_train, Y_test = Y[train_index], Y[test_index] 

        model.fit(X_train, Y_train)
        viz = RocCurveDisplay.from_estimator(model, X_test, Y_test, name = "ROC fold{}".format(i), alpha = 0.5,
                                             lw = 4, ax = ax)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    mean_tpr = np.mean(tprs, axis = 0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    ax.plot(mean_fpr, mean_tpr, color = "blue", label = r"Mean ROC(AUC = %0.2f ± %0.2f)" % (mean_auc, std_auc),
            lw = 8, alpha = 0.9)

    std_tpr = np.std(tprs, axis = 0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color = "grey", alpha = .2, label = "± 1 std. dev.")

    ax.set(xlim = [-0.05, 1.05], ylim = [-0.05, 1.05])
    ax.axis("square")
    ax.grid()  # Dropping grid line 去除网格线
    ax.legend(loc = "lower right")
    ticks = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    ax.set_xlabel("False Positive Rate", labelpad = 15, fontsize = 50, fontweight = "bold") # Distance of line and caption 拉开轴标题与刻度线距离
    ax.set_ylabel("True Positive Rate", labelpad = 15, fontsize = 50, fontweight = "bold")
    ax.set_xticks(ticks, labels = ticks, fontsize = 50, fontweight = "bold") # Scale setting 设置刻度
    ax.set_yticks(ticks, labels = ticks, fontsize = 50, fontweight = "bold")
    plt.setp(ax.spines.values(), linewidth = 4, color = "black") # Line thickness setting 设置边框粗细
    plt.plot([0, 1], [0, 1], linestyle = "--", color = "black", alpha = 0.8, lw = 4) # Diagonal drawing 绘制对角线
    plt.tick_params(axis = "both", length = 10, width = 4)  # Scale thickness and longth setting 设置刻度的粗细和长度
    # plt.savefig(f"{n}-fold ROC.png", dpi = 300)  # Save pic before show() 保存图片，在show()之前

    plt.show()

#### Feature Engineering 特征筛选

In [ ]:
GB = GradientBoostingClassifier(random_state = 42)

for fn in range(10, 41, 10):
    prin_fp = mrmr_selection(fp_tra, cate_tra, fn)
    fe_tra = fp_tra.loc[:,prin_fp] 
    sel_pef = []
    for scoring in criterion:
        answer = cross_val_score(GB, fe_tra, np.array(cate_tra).ravel(), cv = SKFold, scoring = scoring)
        sel_pef.append(answer.mean().round(3))

prin_fp = mrmr_selection(fp_tra, cate_tra, 11)
fe_tra = fp_tra.loc[:,prin_fp]

#### Azo Dataset 偶氮数据集

In [ ]:
fe_all = esm_fp3.loc[:,prin_fp]

fe_azo = fe_all.iloc[azos,]
fe_naz = fe_all.drop(fe_all.index[azos])
cate_azo = [val for ind, val in enumerate(cate) if ind in azos]
cate_naz = [val for ind, val in enumerate(cate) if ind not in azos]

GB = GradientBoostingClassifier(random_state = 42, learning_rate = 0.1, min_samples_split =13, 
                                min_samples_leaf = 1,max_depth = 5, max_features = 3, subsample = 0.8, 
                                n_estimators = 40)

#### Azo Dataset Stratified CV 偶氮数据集分层交叉验证

In [ ]:
X_azo = fe_azo.astype("float") 
Y_azo = np.array(cate_azo).ravel()

GB = GradientBoostingClassifier(random_state = 54, learning_rate = 0.1, min_samples_split =13, min_samples_leaf = 1,
                                max_depth = 5, max_features = 3, subsample = 0.8, n_estimators = 40)

azo_10 = skfold_perf(10, GB, X_azo, Y_azo)
print("Acc:{}, F1 score:{}, AUC:{}".format(azo_10[0], azo_10[1], azo_10[2]))
print("10-fold confusion matrix:")
cm_drawing(10, GB, X_azo, Y_azo)

#### Anthraquinone Dataset 蒽醌数据集

In [ ]:
fe_anq = fe_all.iloc[anqs,]
fe_naq = fe_all.drop(fe_all.index[anqs])
cate_anq = [val for ind, val in enumerate(cate) if ind in anqs]
cate_naq = [val for ind, val in enumerate(cate) if ind not in anqs]

GB = GradientBoostingClassifier(random_state = 42, learning_rate = 0.1, min_samples_split =13, 
                                min_samples_leaf = 1,max_depth = 5, max_features = 3, subsample = 0.8, 
                                n_estimators = 40)

#### Anthraquinone Dataset Stratified CV 蒽醌数据集分层交叉验证

In [ ]:
X_anq = fe_anq.astype("float") 
Y_anq = np.array(cate_anq).ravel()

GB = GradientBoostingClassifier(random_state = 52, learning_rate = 0.1, min_samples_split =13, 
                            min_samples_leaf = 1,max_depth = 5, max_features = 3, subsample = 0.8, 
                            n_estimators = 40)
anq_10 = skfold_perf(10, GB, X_anq, Y_anq)
print("Acc:{}, F1 score:{}, AUC:{}".format(anq_10[0], anq_10[1], anq_10[2]))

print("10-fold confusion matrix:")
cm_drawing(10, GB, X_anq, Y_anq)

#### Others Dataset Stratified CV 其他染料数据集分层交叉验证

In [ ]:
fe_oth = fe_all.iloc[oths,]
cate_oth = [val for ind, val in enumerate(cate) if ind in oths]

X_oth = fe_oth.astype("float") 
Y_oth = np.array(cate_oth).ravel()

GB = GradientBoostingClassifier(random_state = 42, learning_rate = 0.1, min_samples_split =13, 
                            min_samples_leaf = 1,max_depth = 5, max_features = 3, subsample = 0.8, 
                            n_estimators = 40)

oth_05 = skfold_perf(5, GB, X_oth, Y_oth)
print("Acc:{}, F1 score:{}, AUC:{}".format(oth_05[0], oth_05[1], oth_05[2]))

print("05-fold confusion matrix:")
cm_drawing(10, GB, X_oth, Y_oth)